## Imports

In [2]:
# %matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import seaborn as sns
from subprocess import call
from collections import defaultdict
import pandas_profiling

## Constants

In [3]:
TRAINING_DATA = '../datasets/fatal.csv'
d = defaultdict(LabelEncoder)

## Wrangle Data

In [15]:
datapoints = pd.read_csv(TRAINING_DATA)
profiling_datapoints = datapoints
datapoints = datapoints.drop('Unnamed: 0', axis=1)\
                       .drop('name', axis=1)\
                       .drop('date', axis=1)\
                       .drop('race', axis=1)\
                       .drop('id', axis=1)\
                       .dropna()
input_features = datapoints.drop('manner_of_death', axis=1)
expected = datapoints['manner_of_death']


In [5]:
input_features.head()


,armed,age,gender,city,state,signs_of_mental_illness,threat_level,flee,body_camera,state_pop,Ethnicity
0,gun,53.0,M,Shelton,WA,True,attack,Not fleeing,False,7073146,0
1,gun,47.0,M,Aloha,OR,False,attack,Not fleeing,False,3982267,1
2,unarmed,23.0,M,Wichita,KS,False,other,Not fleeing,False,2898292,2
3,toy weapon,32.0,M,San Francisco,CA,True,attack,Not fleeing,False,38654206,1
4,nail gun,39.0,M,Evans,CO,False,attack,Not fleeing,False,5359295,2


In [6]:
expected.head()

0                shot
1                shot
2    shot and Tasered
3                shot
4                shot
Name: manner_of_death, dtype: object

Encode Data:

In [7]:
# input_features_encoded = input_features.apply(LabelEncoder().fit_transform)
input_features_encoded = input_features.apply(lambda x: d[x.name].fit_transform(x))

input_features_encoded.head()

,armed,age,gender,city,state,signs_of_mental_illness,threat_level,flee,body_camera,state_pop,Ethnicity
0,28,43,1,1440,47,1,0,2,0,38,0
1,28,37,1,24,37,0,0,2,0,24,1
2,72,12,1,1699,16,0,1,2,0,17,2
3,71,21,1,1389,4,1,0,2,0,50,1
4,50,29,1,474,5,0,0,2,0,29,2


## Create/Train Classifier

Create training sets:

In [8]:
X_train, X_test, y_train, y_test = train_test_split(input_features_encoded, expected, random_state=1)

Create model and train:

In [9]:
model = tree.DecisionTreeClassifier(max_depth=3)
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

## Visualize Results

In [10]:
y_predict = model.predict(X_test)
"Accuracy Score: " + str(accuracy_score(y_test.as_matrix(), y_predict))


'Accuracy Score: 0.948905109489'

Confusion Matrix:

In [11]:
pd.DataFrame(
        confusion_matrix(y_test, y_predict),
        columns=['Predicted 1', 'Predicted 2'],
        index=['Actual 1', 'Actual 2']
    )

,Predicted 1,Predicted 2
Actual 1,780,0
Actual 2,42,0


Generate tree map:

In [12]:
tree.export_graphviz(model, out_file='manner_of_death_classification.dot', feature_names=input_features.columns)
call(['dot', '-T', 'png', './manner_of_death_classification.dot', '-o', './manner_of_death_classification.png'])

0

Translate Split 'state_pop' Nodes of Interest:

In [13]:
le = LabelEncoder()
le.fit(input_features['armed'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_name_mapping

{'BB gun': 0,
 'Taser': 1,
 'air conditioner': 2,
 'ax': 3,
 'baseball bat': 4,
 'baseball bat and bottle': 5,
 'baseball bat and fireplace poker': 6,
 'baton': 7,
 'bayonet': 8,
 'bean-bag gun': 9,
 'beer bottle': 10,
 'blunt object': 11,
 'box cutter': 12,
 'brick': 13,
 'carjack': 14,
 'chain': 15,
 'chain saw': 16,
 'chainsaw': 17,
 'chair': 18,
 "contractor's level": 19,
 'cordless drill': 20,
 'crossbow': 21,
 'crowbar': 22,
 'fireworks': 23,
 'flagpole': 24,
 'flashlight': 25,
 'garden tool': 26,
 'glass shard': 27,
 'gun': 28,
 'gun and car': 29,
 'gun and knife': 30,
 'gun and sword': 31,
 'guns and explosives': 32,
 'hammer': 33,
 'hand torch': 34,
 'hatchet': 35,
 'hatchet and gun': 36,
 'incendiary device': 37,
 'knife': 38,
 'lawn mower blade': 39,
 'machete': 40,
 'machete and gun': 41,
 'meat cleaver': 42,
 'metal hand tool': 43,
 'metal object': 44,
 'metal pipe': 45,
 'metal pole': 46,
 'metal rake': 47,
 'metal stick': 48,
 'motorcycle': 49,
 'nail gun': 50,
 'oar': 5

# Profiling

In [16]:
pandas_profiling.ProfileReport(profiling_datapoints)

Number of variables,17
Number of observations,3399
Total Missing (%),0.0%
Total size in memory,405.0 KiB
Average record size in memory,122.0 B
Numeric,4
Categorical,10
Boolean,2
Date,0
Text (Unique),0
Rejected,1
